In [5]:
import pandas as pd

In [6]:
import pickle

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import root_mean_squared_error, mean_squared_error

In [9]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcap/02-Experiment-tracking/mlruns/1', creation_time=1716566344394, experiment_id='1', last_update_time=1716566344394, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [10]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [11]:
df_train = read_dataframe("./data/yellow_tripdata_2023-01.parquet")
df_val = read_dataframe("./data/yellow_tripdata_2023-02.parquet")

In [8]:
len(df_train), len(df_val)

(3009173, 2855951)

In [9]:
# df_train.duration.describe

In [12]:
df_train["PU_DO"] = df_train["PULocationID"] + '_' + df_train["PULocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + '_' + df_val["PULocationID"]

In [13]:
df_val ;

In [14]:
categorical = ['PULocationID','DOLocationID'] #["PU_DO"]
#numerical = ['trip_distance']

dv = DictVectorizer()

# train_dicts = df_train[categorical + numerical].to_dict(orient = "records")
train_dicts = df_train[categorical].to_dict(orient = "records")
X_train = dv.fit_transform(train_dicts)


In [15]:
# val_dicts = df_val[categorical + numerical].to_dict(orient = "records")
val_dicts = df_val[categorical].to_dict(orient = "records")
X_val = dv.transform(val_dicts)

In [16]:
target= 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
X_val[0].toarray().shape

(1, 515)

In [16]:
X_train[0].toarray().shape
# X_train[0].toarray()

(1, 515)

In [17]:
# testing model using jan data, and using feb data for validation(testing)
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)


In [18]:
root_mean_squared_error(y_val, y_pred)

7.811817745843695

In [19]:
with open('./models/lin_reg.bin', 'wb' ) as f_out :
    pickle.dump((dv, lr), f_out)

In [20]:
# testing model with its own traning data as validation data
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.649261934850555

In [21]:
# testing model with its traning data as validation data
with mlflow.start_run():

    mlflow.set_tag("developer", "Ganesh")

    mlflow.log_param("train-data-path", "./data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "./data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.001
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

In [17]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [21]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                                                   | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [14:55:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397725960/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.25475                                                                                                                    
[1]	validation-rmse:8.66699                                                                                                                    
[2]	validation-rmse:8.12508                                                                                                                    
[3]	validation-rmse:7.83255                                                                                                                    
[4]	validation-rmse:7.62400                                                                                                                    
[5]	validation-rmse:7.43957                                                                                                                    
[6]	validation-rmse:7.32741                                                                                                             

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [15:09:15] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397725960/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.83505                                                                                                                    
[1]	validation-rmse:6.27115                                                                                                                    
[2]	validation-rmse:6.03178                                                                                                                    
[3]	validation-rmse:5.86015                                                                                                                    
[4]	validation-rmse:5.80065                                                                                                                    
[5]	validation-rmse:5.75429                                                                                                                    
[6]	validation-rmse:5.67157                                                                                                             

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [15:32:54] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397725960/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.80221                                                                                                                    
[1]	validation-rmse:7.99250                                                                                                                    
[2]	validation-rmse:7.28875                                                                                                                    
[3]	validation-rmse:6.97022                                                                                                                    
[4]	validation-rmse:6.35947                                                                                                                    
[5]	validation-rmse:5.98175                                                                                                                    
[6]	validation-rmse:5.89199                                                                                                             

KeyboardInterrupt: 